In [1]:
import pandas as pd
bug_report_raw=pd.read_pickle("Output/allBugReports.pickle")
display(bug_report_raw)

source_code_raw = pd.read_pickle("Output/allSourceCodes.pickle").dropna()
display(source_code_raw)

,fix,text,fixdate,summary,description,project,average_precision
id,,,,,,,
217,[org.apache.commons.collections.map.flat3map.j...,NaN,2006-07-18 22:02:11,Flat3Map.Entry.setValue() overwrites other Ent...,Flat3Map&amp;apos;s Entry objects will overwri...,COLLECTIONS.tar,0.0
214,[org.apache.commons.collections.testextendedpr...,NaN,2006-07-18 22:44:33,ExtendedProperties - field include should be n...,"The field ""include"" in ExtendedProperties is c...",COLLECTIONS.tar,0.0
222,[org.apache.commons.collections.testlistutils....,NaN,2006-08-18 19:01:22,CollectionUtils removeAll is actually retainAll,"The removeAll(Collection collection, Collectio...",COLLECTIONS.tar,0.0
261,[org.apache.commons.collections.map.flat3map.j...,NaN,2007-08-20 14:11:54,Flat3Map.remove() does not return the correct ...,final Flat3Map m = new Flat3Map();\n ...,COLLECTIONS.tar,0.0
264,[org.apache.commons.collections.fasttreemap.java],NaN,2007-08-31 09:39:59,FastTreeMap forgets the comparator,In line 359 and 582 of the current 3.2 release...,COLLECTIONS.tar,0.0
...,...,...,...,...,...,...,...
692,[org.wildfly.security.auth.realm.legacypropert...,NaN,2016-11-02 09:35:48,Add tests for special chars in LegacyPropertie...,Add tests for issue https://issues.jboss.org/b...,ELY.tar,0.0
691,[org.wildfly.security.auth.realm.legacypropert...,NaN,2016-11-02 09:36:13,Elytron properties-realm is not compatible wit...,When users properties file (e.g. mgmt-users.pr...,ELY.tar,0.0
637,[org.wildfly.security.auth.server.serverauthen...,NaN,2016-11-03 15:03:29,No log messages comming from Elytron - permiss...,Elytron is missing any log messages related to...,ELY.tar,0.0


,filename,unprocessed_code,project
0,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS.tar
1,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS.tar
2,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS.tar
3,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS.tar
4,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS.tar
...,...,...,...
63,\gitrepo\src\main\java\org\wildfly\security\ut...,"/*\n * JBoss, Home of Professional Open Source...",ELY.tar
64,\gitrepo\src\main\java\org\wildfly\security\_p...,"/*\n * JBoss, Home of Professional Open Source...",ELY.tar
65,\gitrepo\src\test\java\org\wildfly\security\ma...,"/*\n * JBoss, Home of Professional Open Source...",ELY.tar
66,\gitrepo\src\test\java\org\wildfly\security\ma...,"/*\n * JBoss, Home of Professional Open Source...",ELY.tar


## Cleaning the Data

Snowball Stemmer is used since it is a more powerful stemmer than what is used by the research paper on IRFL.
  
In addition to NLTK's stop words, Python and Java keywords have been removed to provide a better model for analysis. The Python keywords will come from a built-in library called "keyword", while Java language keywords will be extracted from a file provided by the original GitHub source found here: https://github.com/exatoa/Bench4BL/blob/master/scripts/languages.txt
All letters will be turned into lowercases so there would not be any distinction between "Value" and "value", for example.


Initially, joined words such as setValueObject were to be split into its respective words the same way that the research paper did. A method using the Viterbi algorithm was determined to be the most suitable method for splitting words. However, since the dictionary required for this algorithm originates from the NLTK library, it created different words that lost meaning after splitting. Therefore, <b>variable names that are often seen as compound words will not be split for this project.</b> The viterbi algorithm implementation shown below simply serves as an example of why this decision was made. Reference: https://stackoverflow.com/questions/195010/how-can-i-split-multiple-joined-words

In [14]:
import nltk
from collections import Counter
nltk.download('words')

list(string.ascii_lowercase)
def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def word_prob(word): return dictionary[word] / total
dictionary = Counter(words.words())
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

# the following words are samples from the bug report dataset
print("Expected: ['set', 'Value','Object']\nActual: ", viterbi_segment("setValueObject")[0])

Expected: ['set', 'Value','Object']
Actual:  ['set', 'V', 'a', 'lue', 'O', 'b', 'j', 'e', 'c', 't']


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Nicole\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [25]:
import re, string
import pandas as pd
import numpy as np
import keyword
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt 
import html

%matplotlib inline

stemmer = SnowballStemmer("english")

java_keywords = None
with open('languages.txt', 'r') as file:
    java_keywords = file.read().split('\n')

def clean_text(text):
    """
    text: str, returns: str
    """
    # Your solution
    stop_words = list(ENGLISH_STOP_WORDS) + keyword.kwlist + java_keywords
#     print(list(set(stop_words)))

    # This is to remove the double html encodings such as &amp;apos; into simply &apos;.
    # This html encoding is easier to convert into punctuation or special characters for removal.
    text = text.replace("&amp;", '&')
    text = text.replace("\n", '')
    text = html.unescape(text)
    text = text.translate(str.maketrans('','',string.punctuation))
    text = " ".join(text.split())
    
    words = text.split(" ")
    words = [word for word in words if not word in list(set(stop_words))]
    words = [stemmer.stem(word) for word in words]
    text = ' '.join(words)
    return text

print("Before:\n", bug_report_raw["description"][0])
print("\n\nAfter:\n", clean_text(bug_report_raw["description"][0]))


Before:
 Flat3Map&amp;apos;s Entry objects will overwrite other Entry&amp;apos;s values if Entry.setValue() is called on one.  It should only overwrite the Entry at hand.
I&amp;apos;ve looked at the source, and the case statement incorrectly falls through, rather than returning like it should:
Flat3Map.java, lines 646-660:
        public Object setValue(Object value) {
            if (canRemove == false) 
{
                throw new IllegalStateException(AbstractHashedMap.SETVALUE_INVALID);
            }
            Object old = getValue();
            switch (nextIndex) 
{
                case 3: 
                    parent.value3 = value;
                case 2:
                    parent.value2 = value;
                case 1:
                    parent.value1 = value;
            }
            return old;
        }
With this code, if I set the value of the third item in the EntrySet, then all three values are set to the new value.


After:
 flat3map entri object overwrit entri valu